# ACL Checker

## 1. startup.pyの実行

startup.pyを実行し、ライブラリのインポートなどを行う。

In [65]:
%run startup.py

## 2. Batfishコンテナの起動

Batfishコンテナが起動しているか確認し、起動していない場合は起動する。

In [89]:
%%bash
if (docker ps -f "ancestor=batfish/batfish" | grep "0.0.0.0:9996-9997" > /dev/null); then
  echo "Batfish container is already running."
else
　echo "Batfish container is not running yet. Starting now..."
  docker run -d -p 9997:9997 -p 9996:9996 batfish/batfish
fi

Batfish container is already running.


## 3. 通信の設定

以下のセル内の各変数に検証したい通信の情報を格納し、実行する。
HeaderConstraintsのその他の引数は以下を参照。
[https://github.com/batfish/pybatfish/blob/master/pybatfish/datamodel/flow.py#L647-L669](htps://github.com/batfish/pybatfish/blob/master/pybatfish/datamodel/flow.py#L647-L669)

In [125]:
# 送信元IPアドレス
src_ip = "192.168.10.1"
# 宛先IPアドレス
dst_ip = "192.168.20.1"
# アプリケーション
app = "http"
flow = HeaderConstraints(srcIps=src_ip,
                         dstIps=dst_ip,
                         applications=[app])

## 4 ホストの設定

以下のセル内のJSONに送信元および宛先のホスト情報を指定し、実行する。

In [126]:
import json

host_file_dir = "networks/example2/hosts/"
src_host_dict = {
                 "hostname": "src_host",
                 "hostInterfaces": {
                   "eth0": {
                     "name": "eth0",
                     "prefix": str(src_ip + "/24"),
                     }
                   }
                 }

dst_host_dict = {
                 "hostname": "dst_host",
                 "hostInterfaces": {
                   "eth0": {
                     "name": "eth0",
                     "prefix": str(dst_ip + "/24"),
                     }
                   }
                 }

f = open("{dir}src_host.json".format(dir=host_file_dir), "w")
f.write(json.dumps(src_host_dict))
f.close()

f = open("{dir}dst_host.json".format(dir=host_file_dir), "w")
f.write(json.dumps(dst_host_dict))
f.close()

## 3. 設定ファイルのアップロード

設定ファイルをBatfishコンテナへアップロードする(スナップショット名の後にいくつかのログが表示される)。

In [127]:
# ネットワークの名前を指定する
NETWORK_NAME = "datacenter1"
# ネットワークの状態の名前を指定する(current, candidateなど)
SNAPSHOT_NAME = "current"
# 設定ファイルの格納先を指定する
SNAPSHOT_PATH = "networks/example2"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'current'

## 5. 通信の検証

以下のセルを実行し、結果を確認する。

In [134]:
traceroute_ans = bfq.traceroute(
                                headers=flow,
                                startLocation="src_host"
                                #startLocation="enter(core[GigabitEthernet1/0])"
                               ).answer()
display_html(traceroute_ans.frame())

,Flow,Traces,TraceCount
0,192.168.10.1:49152 → 192.168.20.1:80start=src_host,"ACCEPTED1. node: src_hoststeps: FORWARDED(Routes: static [Network: 0.0.0.0/0, Next Hop IP:AUTO/NONE(-1l)]) → TRANSMITTED(eth0)2. node: coresteps: RECEIVED(GigabitEthernet1/0: seg10in) → FORWARDED(Routes: connected [Network: 192.168.20.0/24, Next Hop IP:AUTO/NONE(-1l)]) → TRANSMITTED(GigabitEthernet2/0)3. node: dst_hoststeps: RECEIVED(eth0) → ACCEPTED(InboundStep)",1
